In [2]:
import pandas as pd
from pyhive import hive
from matplotlib import pyplot as plt
host='*****'
username='*****'
password='*****'
port=10000
data_base_name='*****'
#连接hive
conn = hive.Connection(host=host,
                       port=port,
                       auth="CUSTOM",
                       database=data_base_name,
                       username=username,
                       password=password)

#读取数据
sql_order = 'select * from ***** where dt = "2022-06-12" or dt = "2022-06-19"'
train_data = pd.read_sql(sql_order, conn)
# conn.close
print(train_data.info())

             station_id             station_name  7_day_order_count city  \
0      1690751108474928         山东省日照市东港区后村公共快充站                 34  日照市   
1       282572462057520            山东省日照市西站公共充电站                120  日照市   
2      1128048405044944            晋江中骏四季花城迅源充电站                306  泉州市   
3      1690363616651312   安徽省芜湖市鸠江区南翔万商B地块停车场充电站                 83  芜湖市   
4      1689773953083440     沈海高速朴里服务区公共充电站（海口方向）                493  泉州市   
...                 ...                      ...                ...  ...   
63354   283028592917552               万马爱充南大科学园站                428  南京市   
63355  1689414386683952            宁波市镇海山外山酒楼庄市店                  1  宁波市   
63356   565107159572944  开迈斯充电站(宁波市九龙湖开元度假村交直流站)                  0  宁波市   
63357   353171675966256         山西省太原市清徐县供电公司充电站                352  太原市   
63358  1690181706004944          阳江市阳春陂面供电所公共充电站                  5  阳江市   

      city_code district district_code  position_type  operate_construction  \
0       

In [3]:
train_data.rename(columns={"7_day_order_count":"week_day_order_count"},inplace=True)
train_data = train_data.drop(train_data[train_data.week_day_order_count == 0].index)

In [4]:
pd.set_option('display.max_rows', None)
train_data.isnull().sum()

station_id                       0
station_name                     0
week_day_order_count             0
city                             0
city_code                       54
district                         2
district_code                    0
position_type                43145
operate_construction             0
station_type                     0
equipment_count                  0
ac_count                         0
dc_count                         0
power_count                     35
fee_info                       694
payment                      51608
d_house_count                    0
d_work_count                     0
d_shopping_count                 0
d_traffic_count                  0
d_scenic_count                   0
d_populace                       0
d_area                           0
d_populace_density               0
c_total_invertory_count          0
c_2021_total_count               0
c_2021_down_year                 0
c_tsl_total_count                0
c_wl_total_count    

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer, mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
for vals in range(20, 0, -2):
    vals = vals/100
    # print(train_data.corr()["week_day_order_count"].sort_values(ascending = False))
    sort_feature = dict(train_data.corr()["week_day_order_count"].sort_values(ascending = False))
    imp_feature = [key for key, val in sort_feature.items() if val >= vals][1:]
    # print(imp_feature)
    #数据集打乱

    train_data = shuffle(train_data,random_state = 19)
    week_day_order_count = train_data.week_day_order_count
    x_train, x_test, y_train, y_test = train_test_split(train_data[imp_feature],week_day_order_count, test_size=0.2, random_state = 10)
    rf_model = RandomForestRegressor(n_estimators=320, random_state=0)
    rf_model.fit(x_train, y_train)
    rf_pred = rf_model.predict(x_test)
    print(vals, mean_squared_error(y_test, rf_pred) ** 0.5)

0.2 283.4008028089081
0.18 258.918730704718
0.16 226.75736148009761
0.14 240.12567319003378
0.12 206.87108242307914
0.1 294.85255402637546
0.08 231.93583114975863
0.06 227.8423605311907
